# Build and deploy the docker container
Ensure this notebook is running above the "container" folder containing the dockerfile.

In [1]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-quickslack

cd container

chmod +x model/train
chmod +x model/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-1 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Couldn't find program: 'sh'


In [2]:
# S3 prefix
prefix = 'speedcow'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [ ]:
# # tar the model file:
# !tar -czvf model.tar.gz ./word2vec_2.model

In [3]:
from sagemaker.session import Session

model_data = Session().upload_data(path='model.tar.gz', key_prefix='model')
print(model_data)

s3://sagemaker-us-east-1-639634733305/model/model.tar.gz


In [4]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [5]:
# generating the image path
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-word2vec:latest'.format(account, region)

In [6]:
# to deploy the model, you need to have the model created based on your model artifacts, 
# create an endpoint configuration and then create the endpoint based on the two.
# first let's create the model

client = boto3.client('sagemaker')
ModelName='TestCx-BYOA' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
first_model = client.create_model(
    ModelName=ModelName,
    PrimaryContainer={
        'Image': image,
        'ModelDataUrl': 's3://sagemaker-us-east-1-639634733305/model/model.tar.gz'    # note that the model.tar.gz file is a tarball of our word2vec_2.model file
        },
    ExecutionRoleArn=role)

In [7]:
# create the endpoint configuration
endpoint_config_name = 'TestCx-BYOA-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m4.xlarge',
        'InitialVariantWeight':1,
        'InitialInstanceCount':1,
        'ModelName':ModelName,
        'VariantName':'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

TestCx-BYOA-2020-01-22-22-09-54
Endpoint Config Arn: arn:aws:sagemaker:us-east-1:639634733305:endpoint-config/testcx-byoa-2020-01-22-22-09-54


In [8]:
# create the endpoint
endpoint_name = 'TestCx-BYOA-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

TestCx-BYOA-2020-01-22-22-09-54
arn:aws:sagemaker:us-east-1:639634733305:endpoint/testcx-byoa-2020-01-22-22-09-54


In [9]:
from sagemaker.predictor import csv_serializer, json_deserializer, csv_deserializer

In [10]:
from sagemaker.predictor import RealTimePredictor

predictor = RealTimePredictor(endpoint = endpoint_name, sagemaker_session=sess, serializer=csv_serializer, deserializer=csv_deserializer)

In [11]:
# client.delete_endpoint(EndpointName=endpoint_name)

In [12]:
#dir(first_model)

In [18]:
coen_bros = "0116282,2042568,1019452,1403865,190590,138524,335245,477348,887883,101410"


In [14]:
def run_predictions(arr_input, batch_size):
    predictions=[]
    
    if type(arr_input) != np.ndarray:
        arr_input = np.array(arr_input)
        
    for arr in np.array_split(arr_input, batch_size):
        if arr.size > 0:
            print("Shape:{0}".format(arr.shape))
            resule = predictor.predict(arr)
            result = result.decode("utf-8")
            result = result.split(',')
            predictions += [np.expm1(float(r)) for r in result]
            
    return predictions

In [15]:
import pandas as pd
test_df = pd.DataFrame(data=['116282', '2042568', '1019452', '1403865', 
                                        '190590', '138524', '335245', '477348', 
                                        '887883', '101410'])

In [19]:
print(predictor.predict(coen_bros))

[["('970922', 0.9125374555587769)"], ["('1121964', 0.9035829305648804)"], ["('468442', 0.8992078900337219)"], ["('1326954', 0.8989676237106323)"], ["('353671', 0.898465633392334)"], ["('1328913', 0.8974928855895996)"], ["('925259', 0.8972901701927185)"]]


In [ ]:
test_input

In [ ]:
import pandas as pd
test_df = pd.DataFrame(data=['116282', '2042568', '1019452', '1403865', 
                                        '190590', '138524', '335245', '477348', 
                                        '887883', '101410'])
test_input = ['116282', '2042568', '1019452', 
             '1403865', '190590', '138524', 
             '335245', '477348', '887883', '101410']

In [ ]:
test_df.values

In [ ]:
test_df[0] = test_df[0].astype(str)

In [ ]:
[[str(x) for x in lst] for lst in test_df]

In [ ]:
test_df.dtypes

In [ ]:
predictor.predict(test_input)

In [ ]:
import pandas as pd
df = open('./data/word2vec_test_data.csv', "r+")
df

In [ ]:
predictor = RealTimePredictor(endpoint = "TestCx-BYOA-2020-01-21-22-14-57", sagemaker_session=sess, serializer=csv_serializer)

In [ ]:
data = data['movieid']
data

In [ ]:
data

In [ ]:
map(int, data)

In [ ]:
df = df.to_json()
df

In [ ]:
predictor.predict(df)